In [1]:
import torch
from ksmm_py.layer.kronecker_sparse.interface import KSLinear

In [2]:
patterns = [(6, 64, 64, 1), (1, 768, 192, 2)] 
dim_in = patterns[0][0] * patterns[0][2] * patterns[0][3] # a_2 * c_2 * d_2
dim_in

384

In [11]:
6 * 64 * 64 + 768 * 192 * 2

319488

In [3]:
dim_in2 = patterns[1][0] * patterns[1][2] * patterns[1][3]
dim_in2

384

In [4]:
weights = [torch.randn(*pattern) for pattern in patterns]
batch_size = 196
batch_size_last = True
device = 'cuda'
x = torch.randn((dim_in, batch_size) if batch_size_last else (batch_size, dim_in), 
                dtype=torch.float16, # torch.float16 is fine too
                device=device)

In [5]:
ksl = KSLinear(patterns=patterns, weights=weights, algo='dense', dtype=x.dtype, bs_last=batch_size_last, device=device)

In [6]:
ksl.factors[0], ksl.factors[1]

(Parameter containing:
 tensor([[-1.2334, -0.7241, -1.1797,  ...,  0.0000,  0.0000,  0.0000],
         [-0.3420, -0.2401,  1.4570,  ...,  0.0000,  0.0000,  0.0000],
         [-1.8906,  1.0273, -0.1937,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.3267, -0.6533, -0.4280],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0029,  1.6699, -1.1270],
         [ 0.0000,  0.0000,  0.0000,  ..., -1.0000,  1.8564, -0.3325]],
        device='cuda:0', dtype=torch.float16, requires_grad=True),
 Parameter containing:
 tensor([[-0.3025,  0.0000, -1.0645,  ...,  0.0000, -1.8242,  0.0000],
         [ 0.0000, -0.3081,  0.0000,  ...,  2.7988,  0.0000,  2.3457],
         [-2.6836,  0.0000,  0.5244,  ...,  0.0000, -1.9473,  0.0000],
         ...,
         [ 0.0000,  0.5308,  0.0000,  ..., -1.3379,  0.0000, -0.2208],
         [-0.3862,  0.0000,  0.8008,  ...,  0.0000,  1.3477,  0.0000],
         [ 0.0000, -0.4434,  0.0000,  ...,  1.1406,  0.0000,  0.2358]],
     

In [7]:
w = ksl.get_dense_product()
w, w.shape

(tensor([[-1.0303, 10.1875, -1.1904,  ...,  3.1270, -2.2344,  3.2461],
         [ 5.5547,  7.7383,  0.6050,  ...,  6.9141,  3.6230, -5.7305],
         [-0.8955,  1.2529,  0.7163,  ..., -1.2139, -6.7188, -7.1836],
         ...,
         [ 6.6797, 10.0703,  6.1953,  ..., -0.3054, -0.2661,  3.7930],
         [10.1484,  3.9590,  9.8438,  ..., -4.1328, -7.3945, -8.6328],
         [-2.6680, -1.1221,  7.5742,  ..., -5.4141,  5.0000, -7.3008]],
        device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>),
 torch.Size([1536, 384]))

In [8]:
orig = torch.matmul(ksl.factors[1], ksl.factors[0])
orig, orig.shape

(tensor([[-1.0303, 10.1875, -1.1904,  ...,  3.1270, -2.2344,  3.2461],
         [ 5.5547,  7.7383,  0.6050,  ...,  6.9141,  3.6230, -5.7305],
         [-0.8955,  1.2529,  0.7163,  ..., -1.2139, -6.7188, -7.1836],
         ...,
         [ 6.6797, 10.0703,  6.1953,  ..., -0.3054, -0.2661,  3.7930],
         [10.1484,  3.9590,  9.8438,  ..., -4.1328, -7.3945, -8.6328],
         [-2.6680, -1.1221,  7.5742,  ..., -5.4141,  5.0000, -7.3008]],
        device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>),
 torch.Size([1536, 384]))

In [15]:
ksl.get_weights_size() / orig.numel()

0.5416666666666666

In [10]:
1536 *  384

589824

In [13]:
orig_trans = orig.t()
orig_trans.shape

torch.Size([384, 1536])

In [14]:
orig_trans.numel()

589824

In [59]:
output_mm = torch.matmul(x, orig_trans)
output_mm

tensor([[  32.1250, -185.8750, -144.2500,  ...,   69.5000,  -71.1875,
           86.4375],
        [-186.8750,  153.1250,  116.0625,  ...,   91.1250,  -25.3594,
          -69.9375],
        [-125.5625,  234.6250,  278.5000,  ...,  -25.8594,  -81.9375,
          -84.9375],
        ...,
        [-162.3750,  -29.3594,   20.2031,  ..., -102.5625,  -81.8750,
          -58.8438],
        [-155.8750,   19.8438,  -26.1250,  ...,  179.2500, -134.0000,
          -24.6406],
        [  11.5625,  -55.6562,   73.4375,  ..., -118.1875,   73.8750,
           58.2500]], device='cuda:0', dtype=torch.float16,
       grad_fn=<MmBackward0>)

In [60]:
output_ksl_dense = ksl(x)
output_ksl_dense

tensor([[  32.0938, -185.7500, -144.2500,  ...,   69.4375,  -71.2500,
           86.5625],
        [-186.8750,  153.1250,  116.0000,  ...,   91.0625,  -25.3438,
          -69.8750],
        [-125.6250,  234.6250,  278.5000,  ...,  -25.9062,  -82.0000,
          -84.8750],
        ...,
        [-162.3750,  -29.2812,   20.1875,  ..., -102.6250,  -81.9375,
          -58.8125],
        [-155.8750,   19.8750,  -26.1406,  ...,  179.2500, -133.8750,
          -24.5938],
        [  11.6016,  -55.6875,   73.4375,  ..., -118.1875,   73.8125,
           58.2812]], device='cuda:0', dtype=torch.float16,
       grad_fn=<ViewBackward0>)

In [61]:
torch.allclose(output_mm, output_ksl_dense, atol=1e-2, rtol=1e-2)

False

In [62]:
torch.where(torch.abs(output_mm - output_ksl_dense) > 1e-1)

(tensor([  0,   0,   0,  ..., 195, 195, 195], device='cuda:0'),
 tensor([   1,   14,   24,  ..., 1494, 1525, 1532], device='cuda:0'))

In [5]:
from typing import List, Tuple

def create_butterfly_patterns(n: int) -> List[Tuple[int, int, int, int]]:
    """
    Generates the chain of Kronecker-Sparse patterns for a butterfly
    decomposition of a 2^n x 2^n square matrix.

    A butterfly matrix W of size 2^n x 2^n can be factored into n
    Kronecker-Sparse matrices: W = K_1 * K_2 * ... * K_n.

    The ℓ-th factor, K_ℓ, has a pattern corresponding to (a, b, c, d) where:
    a = 2**(ℓ-1)
    b = 2
    c = 2
    d = 2**(n-ℓ)

    Args:
        n: The power of 2 defining the matrix dimension (2^n x 2^n).
           Must be a positive integer.

    Returns:
        A list of n tuples, where each tuple is an (a, b, c, d) pattern.
        The list is ordered for direct use in libraries like ksmm,
        meaning patterns[0] corresponds to the right-most matrix factor (K_n)
        and patterns[-1] corresponds to the left-most factor (K_1).
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("Input 'n' must be a positive integer.")

    patterns = []
    # The loop variable 'l' corresponds to the index in the matrix product K_1 * ... * K_n
    for l in range(1, n + 1):
        a = 2**(l - 1)
        b = 2
        c = 2
        d = 2**(n - l)
        pattern = (a, b, c, d)
        patterns.append(pattern)

    # Reverse the list to match the convention where the first pattern in the list
    # corresponds to the right-most matrix in the product.
    # W = K_1 * K_2 * ... * K_n
    # patterns = [pattern_for_Kn, ..., pattern_for_K2, pattern_for_K1]
    return patterns[::-1]

# --- Example Usage ---
n = 4  # For a 16x16 matrix (2^4 x 2^4)
try:
    butterfly_chain = create_butterfly_patterns(n)
    print(f"Generating butterfly patterns for a {2**n}x{2**n} matrix (n={n}):\n")
    print(f"Matrix product: W = K_1 * K_2 * K_3 * K_4")
    print("Required chain of patterns (from right-most K_4 to left-most K_1):")
    
    # Print the patterns with their corresponding matrix factor
    for i, p in enumerate(butterfly_chain):
        factor_index = n - i
        print(f"  - Pattern for K_{factor_index}: {p}  (a={p[0]}, b={p[1]}, c={p[2]}, d={p[3]})")

except ValueError as e:
    print(e)



Generating butterfly patterns for a 16x16 matrix (n=4):

Matrix product: W = K_1 * K_2 * K_3 * K_4
Required chain of patterns (from right-most K_4 to left-most K_1):
  - Pattern for K_4: (8, 2, 2, 1)  (a=8, b=2, c=2, d=1)
  - Pattern for K_3: (4, 2, 2, 2)  (a=4, b=2, c=2, d=2)
  - Pattern for K_2: (2, 2, 2, 4)  (a=2, b=2, c=2, d=4)
  - Pattern for K_1: (1, 2, 2, 8)  (a=1, b=2, c=2, d=8)


In [7]:
# --- Example for n=3 (8x8 matrix) ---
n_3 = 4
try:
    butterfly_chain_3 = create_butterfly_patterns(n_3)
    print(f"\nGenerating butterfly patterns for a {2**n_3}x{2**n_3} matrix (n={n_3}):\n")
    print(f"Matrix product: W = K_1 * K_2 * K_3")
    print("Required chain of patterns (from right-most K_3 to left-most K_1):")
    
    for i, p in enumerate(butterfly_chain_3):
        factor_index = n_3 - i
        print(f"  - Pattern for K_{factor_index}: {p}")

except ValueError as e:
    print(e)


Generating butterfly patterns for a 16x16 matrix (n=4):

Matrix product: W = K_1 * K_2 * K_3
Required chain of patterns (from right-most K_3 to left-most K_1):
  - Pattern for K_4: (8, 2, 2, 1)
  - Pattern for K_3: (4, 2, 2, 2)
  - Pattern for K_2: (2, 2, 2, 4)
  - Pattern for K_1: (1, 2, 2, 8)


In [8]:
def create_butterfly_patterns(n: int) -> List[Tuple[int, int, int, int]]:
    """
    Generates the chain of Kronecker-Sparse patterns for a butterfly
    decomposition of a 2^n x 2^n square matrix.

    A butterfly matrix W of size 2^n x 2^n can be factored into n
    Kronecker-Sparse matrices: W = K_1 * K_2 * ... * K_n.

    The ℓ-th factor, K_ℓ, has a pattern corresponding to (a, b, c, d) where:
    a = 2**(ℓ-1)
    b = 2
    c = 2
    d = 2**(n-ℓ)

    Args:
        n: The power of 2 defining the matrix dimension (2^n x 2^n).
           Must be a positive integer.

    Returns:
        A list of n tuples, where each tuple is an (a, b, c, d) pattern.
        The list is ordered for direct use in libraries like ksmm,
        meaning patterns[0] corresponds to the right-most matrix factor (K_n)
        and patterns[-1] corresponds to the left-most factor (K_1).
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("Input 'n' must be a positive integer.")

    patterns = []
    # The loop variable 'l' corresponds to the index in the matrix product K_1 * ... * K_n
    for l in range(1, n + 1):
        a = 2**(l - 1)
        b = 2
        c = 2
        d = 2**(n - l)
        pattern = (a, b, c, d)
        patterns.append(pattern)

    # Reverse the list to match the convention where the first pattern in the list
    # corresponds to the right-most matrix in the product.
    # W = K_1 * K_2 * ... * K_n
    # patterns = [pattern_for_Kn, ..., pattern_for_K2, pattern_for_K1]
    return patterns[::-1]

In [12]:
str(create_butterfly_patterns(4))

'[(8, 2, 2, 1), (4, 2, 2, 2), (2, 2, 2, 4), (1, 2, 2, 8)]'

In [ ]:
def list_to_pattern_string(lst):
    return str(lst)